In [ ]:
#IVAM on Total Dead Kilometers Matrix

from collections import defaultdict
import pandas as pd
import numpy as np
import random

randomlist = []
df_demand = {}
random.seed(17) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(100,150) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand[i] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(17) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,50))).astype(float)

random.seed(17) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(17) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(17) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]

df_cost = df_tdkom.copy(deep=True)
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [ ]:
while(list(df_supply.values()) and list(df_demand.values())):

    # To find the minimum cost row-wise
    df_nr_list = list(df_tdkom.min(axis=1))
    df_nr = {}
    for i in range(len(df_tdkom.index)):
        df_nr[df_tdkom.index[i]] = df_nr_list[i]
    df_nr

    # To find the minimum cost column-wise
    df_nc_list = list(df_tdkom.min(axis=0))
    df_nc = {}
    for i in range(len(df_tdkom.columns)):
        df_nc[df_tdkom.columns[i]] = df_nc_list[i]
    df_nc

    # To find the two lowest values row-wise and penalty cost
    df_pen = []
    if len(df_tdkom.columns)!=1:
        df_pend = df_tdkom.apply(lambda x: x.sort_values().unique(), axis=1)
        for j in df_tdkom.index:
            if (len(df_pend[j])) != 1:
                lst = df_pend[j][1] - df_pend[j][0]
                df_pen.append(lst)
            elif (len(df_pend[j])) == 1: ### To check if there is same value across all rows
                df_pen.append(0)
    else:
        df_pen = df_tdkom.apply(lambda x: x.sort_values().unique()[0], axis=1)

    df_penr = {}
    if (df_tdkom.shape[1] != 1):
        for i in range(len(df_tdkom.index)):
            df_penr[df_tdkom.index[i]] = df_pen[i]
    else:
        for i in df_tdkom.index:
            df_penr[i] = df_pen[i]


    ################################################### *** ##############################################################
    # To find the two lowest values column-wise
    df_penc = []
    if len(df_tdkom.index)!=1:

        df_pendo = df_tdkom.apply(lambda x: x.sort_values().unique(), axis=0)

        if (type(df_pendo) == pd.DataFrame):
            for i in df_tdkom.columns:
                if (len(df_pendo[i])!=1):
                    lstc = df_pendo[i].values[1] - df_pendo[i].values[0]
                    df_penc.append(lstc)
                elif (len(df_pendo[i])==1):
                    df_penc.append(0)
        elif (type(df_pendo) == pd.Series):
            for j in df_tdkom.columns:
                if (len(df_pendo[j])) != 1:
                    lstc = df_pendo[j][1] - df_pendo[j][0]
                    df_penc.append(lstc)
                elif (len(df_pendo[j])) == 1: ### To check if there is same value across all rows
                    df_penc.append(0)

    else:
        df_penc = df_tdkom.apply(lambda x: x.sort_values().unique()[0], axis=0)

    df_penco = {}
    if (df_tdkom.shape[0] != 1):
        for i in range(len(df_tdkom.columns)):
            df_penco[df_tdkom.columns[i]] = df_penc[i]
    else:
        for i in df_tdkom.columns:
            df_penco[i] = df_penc[i]

    r = max(list(df_penr.values()) + list(df_penco.values()))

    lstm = []
    lst = []
    lstr = []
    if(r in df_penco.values() and r in df_penr.values()):
        min_val = []
        idx_list = []
        for i in df_tdkom.index:
            for j in (df_tdkom.columns):
                if (df_penr[i] == r and df_penco[j] == r):
                    if (df_nr[i] <= df_nc[j]):
                        for l in df_tdkom.columns:
                            if (df_tdkom.loc[i,l] == df_nr[i]):
                                #print(df_tdkom.loc[i,l], df_nr[i])
                                min_val.append(df_nr[i])
                                idx_list.append([i,l])
                                #lstm.append([i,l])
                    elif (df_nr[i] > df_nc[j]):
                        for o in df_tdkom.index:
                             if (df_tdkom.loc[o,j] == df_nc[j]):
                                    min_val.append(df_nc[j])
                                    idx_list.append([o,j])
                                    #lstm.append([o,j])

                    lstm.append(idx_list[min_val.index(min(min_val))])


        ###### Allocating to the left most cell, since that' how it identifies
        df_result.loc[lstm[0][0],lstm[0][1]] = min(df_supply[lstm[0][0]],df_demand[lstm[0][1]])
        df_supply[lstm[0][0]] = df_supply[lstm[0][0]] - df_result.loc[lstm[0][0],lstm[0][1]]
        df_demand[lstm[0][1]] = df_demand[lstm[0][1]] - df_result.loc[lstm[0][0],lstm[0][1]]


        if (df_supply[lstm[0][0]] == 0 and df_demand[lstm[0][1]] == 0):
            #print(df_supply[lstm[0][0]], df_demand[lstm[0][1]] )
            df_tdkom = df_tdkom.drop(lstm[0][0], axis = 0)
            del df_supply[lstm[0][0]]

        elif (df_supply[lstm[0][0]] == 0 and df_demand[lstm[0][1]] != 0):
            #print(df_supply[lstm[0][0]], df_demand[lstm[0][1]] )
            df_tdkom = df_tdkom.drop(lstm[0][0], axis = 0)
            del df_supply[lstm[0][0]]

        elif (df_supply[lstm[0][0]] != 0 and df_demand[lstm[0][1]] == 0):
            #print(df_supply[lstm[0][0]], df_demand[lstm[0][1]] )
            df_tdkom = df_tdkom.drop(lstm[0][1], axis = 1)
            del df_demand[lstm[0][1]]


    elif(r in df_penco.values() and r not in df_penr.values()):
        for i in df_tdkom.columns:
            if (df_penco[i] == r):
                for j in (df_tdkom.index):
                    if (df_tdkom.loc[j,i] == df_nc[i]):
                        lst.append([j,i])

        if (len(lst) == 1):
            ###### Allocating to the left most cell, since that' how it identifies
            df_result.loc[lst[0][0],lst[0][1]] = min(df_supply[lst[0][0]],df_demand[lst[0][1]])
            df_supply[lst[0][0]] = df_supply[lst[0][0]] - df_result.loc[lst[0][0],lst[0][1]]
            df_demand[lst[0][1]] = df_demand[lst[0][1]] - df_result.loc[lst[0][0],lst[0][1]]


            if (df_supply[lst[0][0]] == 0 and df_demand[lst[0][1]] == 0):
                #print(df_supply[lst[0][0]], df_demand[lst[0][1]] )
                df_tdkom = df_tdkom.drop(lst[0][0], axis = 0)
                del df_supply[lst[0][0]]

            elif (df_supply[lst[0][0]] == 0 and df_demand[lst[0][1]] != 0):
                #print(df_supply[lst[0][0]], df_demand[lst[0][1]] )
                df_tdkom = df_tdkom.drop(lst[0][0], axis = 0)
                del df_supply[lst[0][0]]

            elif (df_supply[lst[0][0]] != 0 and df_demand[lst[0][1]] == 0):
                #print(df_supply[lst[0][0]], df_demand[lst[0][1]] )
                df_tdkom = df_tdkom.drop(lst[0][1], axis = 1)
                del df_demand[lst[0][1]]

        elif (len(lst)!=1):
            minc = df_tdkom.loc[lst[0][0],lst[0][1]]
            for i in range(len(lst)):
                if (minc <= df_tdkom.loc[lst[i][0],lst[i][1]]):
                    minc = minc
                elif (minc > df_tdkom.loc[lst[i][0],lst[i][1]]):
                    minc = df_tdkom.loc[lst[i][0],lst[i][1]]

            lstn = []
            for i in range(len(lst)):
                if (df_tdkom.loc[lst[i][0],lst[i][1]] == minc):
                    lstn.append([lst[i][0],lst[i][1]])

            df_result.loc[lstn[0][0],lstn[0][1]] = min(df_supply[lstn[0][0]],df_demand[lstn[0][1]])
            df_supply[lstn[0][0]] = df_supply[lstn[0][0]] - df_result.loc[lstn[0][0],lstn[0][1]]
            df_demand[lstn[0][1]] = df_demand[lstn[0][1]] - df_result.loc[lstn[0][0],lstn[0][1]]

            if (df_supply[lstn[0][0]] == 0 and df_demand[lstn[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstn[0][0], axis = 0)
                del df_supply[lstn[0][0]]

            elif (df_supply[lstn[0][0]] == 0 and df_demand[lstn[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lstn[0][0], axis = 0)
                del df_supply[lstn[0][0]]

            elif (df_supply[lstn[0][0]] != 0 and df_demand[lstn[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstn[0][1], axis = 1)
                del df_demand[lstn[0][1]]



    elif (r in df_penr.values() and r not in df_penco.values()):
        for i in df_tdkom.index:
            if (df_penr[i] == r):
                for j in (df_tdkom.columns):
                    if (df_tdkom.loc[i,j] == df_nr[i]):
                        lstr.append([i,j])

        if (len(lstr) == 1):
            df_result.loc[lstr[0][0],lstr[0][1]] = min(df_supply[lstr[0][0]],df_demand[lstr[0][1]])
            df_supply[lstr[0][0]] = df_supply[lstr[0][0]] - df_result.loc[lstr[0][0],lstr[0][1]]
            df_demand[lstr[0][1]] = df_demand[lstr[0][1]] - df_result.loc[lstr[0][0],lstr[0][1]]

            if (df_supply[lstr[0][0]] == 0 and df_demand[lstr[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstr[0][0], axis = 0)
                del df_supply[lstr[0][0]]

            elif (df_supply[lstr[0][0]] == 0 and df_demand[lstr[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lstr[0][0], axis = 0)
                del df_supply[lstr[0][0]]

            elif (df_supply[lstr[0][0]] != 0 and df_demand[lstr[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstr[0][1], axis = 1)
                del df_demand[lstr[0][1]]

        elif (len(lstr)!=1):
            minr = df_tdkom.loc[lstr[0][0],lstr[0][1]]
            for i in range(len(lstr)):
                if (minr <= df_tdkom.loc[lstr[i][0],lstr[i][1]]):
                    minr = minr
                elif (minr > df_tdkom.loc[lstr[i][0],lstr[i][1]]):
                    minr = df_tdkom.loc[lstr[i][0],lstr[i][1]]

            lstp = []
            for i in range(len(lstr)):
                if (df_tdkom.loc[lstr[i][0],lstr[i][1]] == minr):
                    lstp.append([lstr[i][0],lstr[i][1]])

            df_result.loc[lstp[0][0],lstp[0][1]] = min(df_supply[lstp[0][0]],df_demand[lstp[0][1]])
            df_supply[lstp[0][0]] = df_supply[lstp[0][0]] - df_result.loc[lstp[0][0],lstp[0][1]]
            df_demand[lstp[0][1]] = df_demand[lstp[0][1]] - df_result.loc[lstp[0][0],lstp[0][1]]

            if (df_supply[lstp[0][0]] == 0 and df_demand[lstp[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstp[0][0], axis = 0)
                del df_supply[lstp[0][0]]

            elif (df_supply[lstp[0][0]] == 0 and df_demand[lstp[0][1]] != 0):
                df_tdkom = df_tdkom.drop(lstp[0][0], axis = 0)
                del df_supply[lstp[0][0]]

            elif (df_supply[lstp[0][0]] != 0 and df_demand[lstp[0][1]] == 0):
                df_tdkom = df_tdkom.drop(lstp[0][1], axis = 1)
                del df_demand[lstp[0][1]]


In [ ]:
df_result

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6276,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6277,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_final = df_cost.mul(df_result, axis = 0)
df_final.to_numpy().sum()

698611.3097761993

stop = timeit.default_timer()
execution_time = stop - start

print("Program Executed in "+str(execution_time))